# Movie Recommender System

This system recommends you movies based on the previously watched content.

Two datasets(Movies and Credits) have been used for recommending movies from https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata

**1. MOVIE DATASET** 
* budget 
* genres 
* homepage
* id
* keywords
* original language
* original title
* overview	
* popularity	
* production_companies	
* production_countries	
* release_date	
* revenue	
* runtime	
* spoken_languages	
* status	
* tagline	
* title	
* vote_average	
* vote_count

**2. CREDIT DATSESET**
* movie_id	
* title	
* cast	
* crew

**Importing relevant libraries**

In [3]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

**Loading Datasets**

In [4]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'tmdb_5000_movies.csv'

# Exploring data

In [ ]:
movies.shape

In [ ]:
credits.shape

In [ ]:
movies.info()

In [ ]:
movies.head(2)

In [ ]:
credits.info()

In [ ]:
credits.head(2)

**Merging both datasets on title**

In [ ]:
df = movies.merge(credits, on = 'title')

In [ ]:
df.shape

In [ ]:
df.head(1)

# Selecting significant features for the recommendation system

In [ ]:
features = ['movie_id','title','overview','genres','keywords','cast','crew']

In [ ]:
data = df[features]

In [ ]:
data.head(2)

**Checking for missing values**

In [ ]:
data.isnull().sum()

**Dropping rows with any missing value**

In [ ]:
data.dropna(inplace = True)

In [ ]:
#checking for duplicate rows
data.duplicated().sum()

# Data Pre-processing

**Extracting 'name' from the genres and keywords into a list**

In [ ]:
import ast

def extract_(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L 

In [ ]:
data['genres'] = data['genres'].apply(extract_)
data.head(2)

In [ ]:
data['keywords'] = data['keywords'].apply(extract_)
data.head(2)

**Extracting 3 lead actors from the cast into a list**

In [ ]:
def extract3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L 

In [ ]:
data['cast'] = data['cast'].apply(extract3)
data.head(2)

In [ ]:
data['cast'] = data['cast'].apply(lambda x:x[0:3])

**Extracting only director's name from crew** 

In [ ]:
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L 

In [ ]:
data['crew'] = data['crew'].apply(fetch_director)

In [ ]:
data.sample(5)

**Removing space in between words**

In [ ]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [ ]:
data['cast'] = data['cast'].apply(collapse)
data['crew'] = data['crew'].apply(collapse)
data['keywords'] = data['keywords'].apply(collapse)
data['genres'] = data['genres'].apply(collapse)

In [ ]:
data.head(3)

**Converting string into a list for 'overview'**

In [ ]:
data['overview'] = data['overview'].apply(lambda x:x.split())

# CREATING TAGS
* A tag recommender system is a recommender system which recommends tags to the user. In this context, a tag is defined as a word freely added to an object by a user.

In [ ]:
data['tags'] = data['overview'] + data['genres'] + data['keywords'] + data['cast'] + data['crew']

In [ ]:
#FINAL DATAFRAME 
movies_df = data[['movie_id','title', 'tags']]

In [ ]:
# converting list of strings to a single string

movies_df['tags'] = movies_df['tags'].apply( lambda x: " ".join(x))

#changing to lower case letters

movies_df['tags'] = movies_df['tags'].apply(lambda x : x.lower())

In [ ]:
movies_df.head(5)

In [ ]:
movies_df['tags'][0]

# Vectorizing tags

**STEMMING all the tags**
* Stemming is the process of reducing a word to its word stem that affixes to suffixes and prefixes or to the roots of words. 
* For example, ['run', 'running', 'runs'] will be converted to ['run','run', 'run' ]

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
ps.stem('running')

In [ ]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    
    return " ".join(y)

In [ ]:
movies_df['tags'] = movies_df['tags'].apply(stem)

In [ ]:
movies_df.head(1)

**Extracting top 5000 occuring words**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5001,stop_words='english')

In [ ]:
vector = cv.fit_transform(movies_df['tags']).toarray()

In [ ]:
vector.shape

**Using COSINE similarity to recommend similar content**
* Cosine similarity measures the similarity between two vectors of an inner product space. It is measured by the cosine of the angle between two vectors and determines whether two vectors are pointing in roughly the same direction.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)
similarity

**Defining our recommendation function**

In [ ]:
def recommend(movie):
    
    #Extracting the index of the given movie
    index = movies_df[movies_df['title'] == movie].index[0]
    
    #calculating the similarity with each movie and sorting in descending order
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    
    for i in distances[1:6]:
        
        #printing top 5 similar movies
        print(movies_df.iloc[i[0]].title)
        

In [ ]:
recommend('Iron Man')

**Deploying models**

In [ ]:
import pickle
import bz2file as bz2

def compressed_pickle(title, data):

    with bz2.BZ2File(title + '.pbz2', 'w') as f:
        pickle.dump(data, f)

compressed_pickle('movies_dict', movies_df.to_dict())
compressed_pickle('similarity', similarity)